In [2]:
%load_ext autoreload
%autoreload 2

# pip install git2doc
from git2doc import loader
import tiktoken
import polars as pl
import os

In [19]:
# pipeline for get_top_repos() -> pull_code_from_repo()
loader.pipeline_fetch_and_load(
    n_repos=10_000,
    last_n_days=365,
    language="python",
    write_batch_size=100,
)

Querying 10000 repos, this will be done in 12 batches
Query: 833 repos on 2022-06-24 -> 2022-07-24
Query: 833 repos on 2022-07-24 -> 2022-08-23
Query: 833 repos on 2022-08-23 -> 2022-09-22
Query: 833 repos on 2022-09-22 -> 2022-10-22
Query: 833 repos on 2022-10-22 -> 2022-11-21
Query: 833 repos on 2022-11-21 -> 2022-12-21
Query: 833 repos on 2022-12-21 -> 2023-01-20
Query: 833 repos on 2023-01-20 -> 2023-02-19
Query: 833 repos on 2023-02-19 -> 2023-03-21
Query: 833 repos on 2023-03-21 -> 2023-04-20
Query: 833 repos on 2023-04-20 -> 2023-05-20
Query: 833 repos on 2023-05-20 -> 2023-06-19

(0) Processing https://github.com/facebookincubator/AITemplate...
(1) Processing https://github.com/clovaai/donut...
(2) Processing https://github.com/jina-ai/discoart...
(3) Processing https://github.com/Visualize-ML/Book4_Power-of-Matrix...
(4) Processing https://github.com/zas023/JdBuyer...
(5) Processing https://github.com/kuprel/min-dalle...
(6) Processing https://github.com/facundoolano/software-

Exception in thread Thread-566 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 1494, in _readerthread
    buffer.append(fh.read())
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 170: character maps to <undefined>
Exception in thread Thread-568 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C

Failed to pull data from https://github.com/datawhalechina/vced after 3 attempts.
(139) Processing https://github.com/garvita-tiwari/PoseNDF...
(140) Processing https://github.com/TEAM-PYRO-BOTZ/PYRO-RENAME-BOT...
(141) Processing https://github.com/AirportR/FullTclash...
(142) Processing https://github.com/Run1e/STRIKER...
(143) Processing https://github.com/OpenDriveLab/ST-P3...
(144) Processing https://github.com/PwnFunction/v8-randomness-predictor...
(145) Processing https://github.com/vyperlang/titanoboa...
(146) Processing https://github.com/prismformore/Multi-Task-Transformer...
(147) Processing https://github.com/iejMac/video2numpy...
(148) Processing https://github.com/NetEase-GameAI/Face2FaceRHO...
(149) Processing https://github.com/SenZHANG-GitHub/ekf-imu-depth...
(150) Processing https://github.com/geerlingguy/pico-w-garage-door-sensor...
(151) Processing https://github.com/r00tman/NeRF-OSR...
(152) Processing https://github.com/PeterWang512/GANWarping...
(153) Processing 

Exception in thread Thread-18206 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 1494, in _readerthread
    buffer.append(fh.read())
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 252: character maps to <undefined>
Exception in thread Thread-18208 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  Fil

Failed to pull data from https://github.com/HarleysZhang/deep_learning_system after 3 attempts.
(4534) Processing https://github.com/Jiaxin-Ye/TIM-Net_SER...
(4535) Processing https://github.com/AlexTMallen/adaptive-retrieval...
(4536) Processing https://github.com/ChenhongyiYang/GPViT...
(4537) Processing https://github.com/jwills/buenavista...
(4538) Processing https://github.com/dongbo811/AFFormer...
(4539) Processing https://github.com/AlUlkesh/sd_save_intermediate_images...
(4540) Processing https://github.com/OWASP/KubeLight...
(4541) Processing https://github.com/OpenDriveLab/PPGeo...
(4542) Processing https://github.com/wez/govee-lan-hass...
(4543) Processing https://github.com/kakaobrain/flame...
(4544) Processing https://github.com/nicolaballotta/gpt3-wordpress-post-generator...
(4545) Processing https://github.com/MedAziz11/Uptime-Kuma-Web-API...
(4546) Processing https://github.com/SourceDiver42/Ghidra-ChatGPT...
(4547) Processing https://github.com/zxyge/tg_file_download_b

In [ ]:
import sys

def sizeof_fmt(num, suffix='B'):
    """Helper to pretty-print memory sizes"""
    for unit in ['', 'Ki', 'Mi', 'Gi', 'Ti', 'Pi', 'Ei', 'Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

In [3]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string, allowed_special="all"))
    return num_tokens

def num_tokens_wrapper(string: str) -> int:
    try:
        if len(string) > 5_000_000:
            # write string to file
            with open("temp.txt", "w") as f:
                f.write(string)
        return num_tokens_from_string(string, "cl100k_base")
    except Exception as e:
        print(f"Failed to process string: {string}. Error: {e}")
        return None


count_dfs = []
folder = "output_data"
for file in os.listdir(folder):
    if "filedata_20230624174108_16" in file:
        print(f"Reading {file}")
        df = pl.read_parquet(os.path.join(folder, file))
        
        df = df.with_columns(
            pl.col("page_content").apply(num_tokens_wrapper).alias("num_tokens")
        )
        count_dfs.append(df[['num_tokens', 'file_type']])

df = pl.concat(count_dfs)
df.to_pandas()['num_tokens'].sum()


Reading filedata_20230624174108_16.parquet


PanicException: python function failed PanicException: called `Result::unwrap()` on an `Err` value: StackOverflow

PanicException: Unwrapped panic from Python code

In [ ]:
# 6,753,475,449 = 6.75 billion tokens

In [12]:
df = pl.read_parquet("output_data/filedata_20230624174108_16.parquet").to_pandas()

# find the top 10 len page_content row
suspect_pages = df.sort_values("page_content", key=lambda s: s.str.len(), ascending=False).head(10)['page_content'].values
df.sort_values("page_content", key=lambda s: s.str.len(), ascending=False).head(10)

,page_content,file_path,file_name,file_type,repo_name
4976,x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x6...,dataset_generation\dataset_sample\train\self\i...,input.arch_emb,.arch_emb,https://github.com/OSUSecLab/SymLM
4985,"push rbp mov rbp , rsp sub rsp , hexvar mov qw...",dataset_generation\dataset_sample\train\self\i...,input.static,.static,https://github.com/OSUSecLab/SymLM
4977,## ## ## ## ## ## ## ## ## 00 ## ## ## ## ## #...,dataset_generation\dataset_sample\train\self\i...,input.byte1,.byte1,https://github.com/OSUSecLab/SymLM
4982,## ## ## ## ## ## ## ## ## 00 ## ## ## ## ## #...,dataset_generation\dataset_sample\train\self\i...,input.byte4,.byte4,https://github.com/OSUSecLab/SymLM
4981,## ## ## ## ## ## ## ## ## 00 ## ## ## ## ## #...,dataset_generation\dataset_sample\train\self\i...,input.byte3,.byte3,https://github.com/OSUSecLab/SymLM
4979,## ## ## ## ## ## ## ## ## 00 ## ## ## ## ## #...,dataset_generation\dataset_sample\train\self\i...,input.byte2,.byte2,https://github.com/OSUSecLab/SymLM
4983,0 0 1 1 1 1 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 ...,dataset_generation\dataset_sample\train\self\i...,input.inst_pos_emb,.inst_pos_emb,https://github.com/OSUSecLab/SymLM
4758,                       ...,data_bin\self\arch_emb\train.bin,train.bin,.bin,https://github.com/OSUSecLab/SymLM
4764,               \t \n     \r  ...,data_bin\self\op_pos_emb\train.bin,train.bin,.bin,https://github.com/OSUSecLab/SymLM
4980,0 1 0 1 2 3 0 1 2 3 0 1 2 3 4 5 6 7 8 9 0 1 2 ...,dataset_generation\dataset_sample\train\self\i...,input.op_pos_emb,.op_pos_emb,https://github.com/OSUSecLab/SymLM


In [11]:
for page in suspect_pages:
    print(f"{page[:100]}\nPage length: {len(page)}")
    num_tokens_wrapper(page)

x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 x64 
Page length: 29982576
push rbp mov rbp , rsp sub rsp , hexvar mov qword ptr [ rbp - hexvar ] , rdi mov rax , qword ptr [ r
Page length: 28139074
## ## ## ## ## ## ## ## ## 00 ## ## ## ## ## ## 00 ## ## ## ## ## ## ## ## ## ## ## 00 ## ## ## ## 0
Page length: 22497791
## ## ## ## ## ## ## ## ## 00 ## ## ## ## ## ## 00 ## ## ## ## ## ## ## ## ## ## ## 00 ## ## ## ## 0
Page length: 22497791
## ## ## ## ## ## ## ## ## 00 ## ## ## ## ## ## 00 ## ## ## ## ## ## ## ## ## ## ## 00 ## ## ## ## 0
Page length: 22497791
## ## ## ## ## ## ## ## ## 00 ## ## ## ## ## ## 00 ## ## ## ## ## ## ## ## ## ## ## 00 ## ## ## ## 0
Page length: 22497791
0 0 1 1 1 1 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 6 6 6 6 7 7 8 8 8 8 8 8 8 8 8 8 
Page length: 20167256
                                                  
Page length: 15056442


PanicException: called `Result::unwrap()` on an `Err` value: StackOverflow